# Experiment

In [1]:
# Script for generate  experiments
# Imports
from clases.red_rddas_model import RedRddasModel
import ray
import time

# Ray Configurations
ray.init(num_cpus=12, log_to_driver=False)

# Experiment 1
print("Experiment 1")
print("============")

# Variable Parameters
n_rddas_min = 3
n_rddas_max = 10

# Fixed Parameters
n_of_variables_rdda = 5
n_of_signals_rdda = 2
n_exit_variables = 2
n_clauses_function = 2
type_network = "ALEATORY"
# this name has to be unique
unique_path = "21_01_2022"

# List of Result for the Experiments
l_res_experiment = []

v_n_network = 1
for n_of_rdds in range(n_rddas_min, n_rddas_max + 1):
    print("Number of Network:", v_n_network)
    print("-------------------------------")

    path = "files/" + unique_path + "_" + str(n_of_rdds) + "_" + str(n_of_variables_rdda) + "_" + str(n_of_signals_rdda) \
           + "_" + str(n_exit_variables) + "_" + str(n_clauses_function)

    # generate the RDDAs of the Network of RDDAs
    print("generating the Network of RDDAs ...")
    oRedRddasModel = RedRddasModel(n_of_rdds, n_of_variables_rdda, n_of_signals_rdda, n_exit_variables,
                                   n_clauses_function)

    # Generate the RDDs
    print("generating the rdds ...")
    oRedRddasModel.generate_rddas(type_network=type_network)

    # # Save the Network of RDDAs in a Pickle file
    # RedRddasModel.save_file_pickle(oRedRddasModel, path)
    # path += ".pickle"
    #
    # print("=======================================================")
    # print("The Network of RDDAs is saved in: ", path)

    # Show the Network of RDDAs
    # oRedRddasModel.show()

    # Calculate the Attractors by RDDA and by Signal
    result = RedRddasModel.find_attractors_rddas_ray.remote(oRedRddasModel)
    oRedRddasModel = ray.get(result)

    # Calculate the Attractors by RDDA and by Signal
    v_begin_0 = time.time()
    result = RedRddasModel.calculation_compatible_pairs.remote(oRedRddasModel)
    oRedRddasModel = ray.get(result)
    v_end_0 = time.time()
    v_time_0 = v_end_0 - v_begin_0

    # Calculate the Attractors by RDDA and by Signal with iterative Method
    v_begin_1 = time.time()
    # result = RedRddasModel.assembly_attractor_fields_iterative.remote(oRedRddasModel)
    # oRedRddasModel = ray.get(result)
    v_end_1 = time.time()
    v_time_1 = v_end_1 - v_begin_1

    v_begin_2 = time.time()
    # Calculate the Attractors by RDDA and by Signal with optimized Method
    result = RedRddasModel.assembly_attractor_fields_optimized.remote(oRedRddasModel)
    oRedRddasModel = ray.get(result)
    v_end_2 = time.time()
    v_time_2 = v_end_2 - v_begin_2

    res_dict = {"n_network": v_n_network,
                "n_rdds": n_of_rdds,
                "n_rdda_attractors": len(oRedRddasModel.d_global_rdda_attractor.items()),
                "n_attractor_fields": len(oRedRddasModel.attractor_fields),
                "t_comp_paris_method": v_time_0,
                # "t_iterative_method": v_time_1,
                "t_optimized_method": v_time_2,
                }
    l_res_experiment.append(res_dict)
    v_n_network = v_n_network + 1

for v_reg_dict in l_res_experiment:
    # Show the metrics
    print("===========================================")
    print("Result of Network:", v_reg_dict["n_network"])
    print("===========================================")
    print("Number of RDDs:", v_reg_dict["n_rdds"])
    print("Number of Attractors :", v_reg_dict["n_rdda_attractors"])
    print("Number of Attractor Fields:", v_reg_dict["n_attractor_fields"])
    print("Duration Compatible Pairs Method :", v_reg_dict["t_comp_paris_method"])
    # print("Duration Iterative Method :", v_reg_dict["t_iterative_method"])
    print("Duration Optimized Method:", v_reg_dict["t_optimized_method"])

print("END EXPERIMENT")

Experiment 1
Number of Network: 1
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
RDDA :  1
VARIABLES :  [1, 2, 3, 4, 5]
RDDA :  2
VARIABLES :  [6, 7, 8, 9, 10]
RDDA :  3
VARIABLES :  [11, 12, 13, 14, 15]
Generating the True Table
A ∨ B
{'6': 'A', '10': 'B'}
{'A': True, 'B': True}
A ∨ B
{'6': 'A', '10': 'B'}
{'A': True, 'B': False}
A ∨ B
{'6': 'A', '10': 'B'}
{'A': False, 'B': True}
A ∨ B
{'6': 'A', '10': 'B'}
{'A': False, 'B': False}
Network Input : 1
Network Output : 2
Variables : [6, 10]
Name Variable : 16
Coupling Function : 6 ∨ 10
Truth Table: {'11': '1', '10': '1', '01': '1', '00': '0'}
Generating the True Table
A ∨ B
{'3': 'A', '4': 'B'}
{'A': True, 'B': True}
A ∨ B
{'3': 'A', '4': 'B'}
{'A': True, 'B': False}
A ∨ B
{'3': 'A', '4': 'B'}
{'A': False, 'B': True}
A ∨ B
{'3': 'A', '4': 'B'}
{'A': False, 'B': False}
Network Input : 2
Network Output : 1
Variables : [3, 4]
Name Variable : 17
Coupling Function : 3 ∨ 4
Truth Table: {'11': '1', 

Number of Network: 5
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
RDDA :  1
VARIABLES :  [1, 2, 3, 4, 5]
RDDA :  2
VARIABLES :  [6, 7, 8, 9, 10]
RDDA :  3
VARIABLES :  [11, 12, 13, 14, 15]
RDDA :  4
VARIABLES :  [16, 17, 18, 19, 20]
RDDA :  5
VARIABLES :  [21, 22, 23, 24, 25]
RDDA :  6
VARIABLES :  [26, 27, 28, 29, 30]
RDDA :  7
VARIABLES :  [31, 32, 33, 34, 35]
Generating the True Table
A ∨ B
{'22': 'A', '25': 'B'}
{'A': True, 'B': True}
A ∨ B
{'22': 'A', '25': 'B'}
{'A': True, 'B': False}
A ∨ B
{'22': 'A', '25': 'B'}
{'A': False, 'B': True}
A ∨ B
{'22': 'A', '25': 'B'}
{'A': False, 'B': False}
Network Input : 1
Network Output : 5
Variables : [22, 25]
Name Variable : 36
Coupling Function : 22 ∨ 25
Truth Table: {'11': '1', '10': '1', '01': '1', '00': '0'}
Generating the True Table
A ∨ B
{'12': 'A', '14': 'B'}
{'A': True, 'B': True}
A ∨ B
{'12': 'A', '14': 'B'}
{'A': True, 'B': False}
A ∨ B
{'12': 'A', '14': 'B'}
{'A': False, 'B': True}
A ∨

Number of Network: 7
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
RDDA :  1
VARIABLES :  [1, 2, 3, 4, 5]
RDDA :  2
VARIABLES :  [6, 7, 8, 9, 10]
RDDA :  3
VARIABLES :  [11, 12, 13, 14, 15]
RDDA :  4
VARIABLES :  [16, 17, 18, 19, 20]
RDDA :  5
VARIABLES :  [21, 22, 23, 24, 25]
RDDA :  6
VARIABLES :  [26, 27, 28, 29, 30]
RDDA :  7
VARIABLES :  [31, 32, 33, 34, 35]
RDDA :  8
VARIABLES :  [36, 37, 38, 39, 40]
RDDA :  9
VARIABLES :  [41, 42, 43, 44, 45]
Generating the True Table
A ∨ B
{'29': 'A', '30': 'B'}
{'A': True, 'B': True}
A ∨ B
{'29': 'A', '30': 'B'}
{'A': True, 'B': False}
A ∨ B
{'29': 'A', '30': 'B'}
{'A': False, 'B': True}
A ∨ B
{'29': 'A', '30': 'B'}
{'A': False, 'B': False}
Network Input : 1
Network Output : 6
Variables : [29, 30]
Name Variable : 46
Coupling Function : 29 ∨ 30
Truth Table: {'11': '1', '10': '1', '01': '1', '00': '0'}
Generating the True Table
A ∨ B
{'20': 'A', '18': 'B'}
{'A': True, 'B': True}
A ∨ B
{'20': 'A', '18

Result of Network: 1
Number of RDDs: 3
Number of Attractors : 17
Number of Attractor Fields: 18
Duration Compatible Pairs Method : 0.012365579605102539
Duration Optimized Method: 0.006114006042480469
Result of Network: 2
Number of RDDs: 4
Number of Attractors : 33
Number of Attractor Fields: 26
Duration Compatible Pairs Method : 0.027016878128051758
Duration Optimized Method: 0.020645618438720703
Result of Network: 3
Number of RDDs: 5
Number of Attractors : 29
Number of Attractor Fields: 0
Duration Compatible Pairs Method : 0.018250226974487305
Duration Optimized Method: 0.002323627471923828
Result of Network: 4
Number of RDDs: 6
Number of Attractors : 35
Number of Attractor Fields: 22
Duration Compatible Pairs Method : 0.02092885971069336
Duration Optimized Method: 0.022921085357666016
Result of Network: 5
Number of RDDs: 7
Number of Attractors : 57
Number of Attractor Fields: 0
Duration Compatible Pairs Method : 0.041362762451171875
Duration Optimized Method: 0.004021406173706055
Res

# Analysing the result

### Time vs RDDAs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Fill the np.array
np_titles = np.array(list(l_res_experiment[0].keys()))
v_reg_dict["n_network"]
v_reg_dict["n_rdds"]

for v_network in l_res_experiment:
    
    for v_title in np_titles:
        print(v_network["n_network"])
    
matrix

In [ ]:


# x =
# y =
#
# fig, ax = plt.subplots()
# ax.plot(x, y)
# plt.show()


In [ ]:


# x =
# y =
#
# fig, ax = plt.subplots()
# ax.plot(x, y)
# plt.show()
